# Tracking Eigenvalues: The Shift-Invert Power Method

### 1. The Strategy
When studying how a Hamiltonian $H(\lambda)$ behaves as we vary a parameter $\lambda$, we don't need to re-diagonalize the whole matrix at every step. Instead, we use a **"tracking" strategy**:

* **The Anchor:** We start with a full diagonalization for an initial $\lambda_0$ to get our baseline energy $E(\lambda_0)$.
* **The Follow-up:** As we shift $\lambda$ slightly, we assume the new energy will be close to the previous one. We use the previous energy as a "guess" or "shift", denoted $\tilde{E}(\lambda)$.

---

### 2. Why Shift-Invert?
The goal is to force the Power Method to ignore the edges of the spectrum and focus exactly on the energy level we are tracking. We do this by applying the iteration to the inverse operator:

$$[H - \tilde{E}(\lambda)I]^{-1} |\psi_n\rangle = \eta |\psi_{n+1}\rangle$$

In this setup, the closer the actual energy $E$ is to our guess $\tilde{E}$, the more the term $1/(E - \tilde{E})$ blows up, making that specific state dominant. This allows the algorithm to converge very quickly to the state we are following.



---

### 3. Avoiding Matrix Inversion
In practice, explicitly calculating the inverse matrix $[H - \tilde{E}I]^{-1}$ is **extremely long and computationally expensive**, especially for large systems. 

To get around this, we treat each step as a linear system to solve:
$$(H - \tilde{E}I) |w\rangle = |\psi_n\rangle$$

We use the **MINRES** algorithm to find $|w\rangle$. It's a clever iterative solver that finds the solution without ever needing to actually invert the matrix, saving a massive amount of time and memory.

---

### 4. Getting the Results Back
Once the iteration settles, we have successfully "captured" the state. We can then easily recover the physical properties of the system:

* **The Eigenvector:** The converged $|\psi\rangle$ is the new state of your Hamiltonian.
* **The Eigenvalue:** We find the energy $E$ simply by calculating the **Rayleigh quotient**:

$$E = \frac{\langle\psi|H|\psi\rangle}{\langle\psi|\psi\rangle}$$

---

# Implementation

In [7]:
import os
import pickle
import time

import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as dense_linalg
import scipy.sparse as sp
import scipy.sparse.linalg as sp_linalg
from joblib import Parallel, delayed
from scipy import sparse
from threadpoolctl import threadpool_limits
from tqdm.notebook import tqdm


### Conversion $C^{n} \to R^{2n}$

As minres only works with real matrixes, we need to add some function to convert our complex operators (if they are) to real ones.

In [8]:
def to_real_basis(H_complex):
    if sp.issparse(H_complex):
        return sp.bmat([[H_complex.real, -H_complex.imag], [H_complex.imag, H_complex.real]], format="csr")
    else:
        return np.block([[H_complex.real, -H_complex.imag], [H_complex.imag, H_complex.real]])


def vec_to_real(v_complex):
    return np.concatenate((v_complex.real, v_complex.imag))


def vec_to_complex(v_real):
    n = len(v_real) // 2
    return v_real[:n] + 1j * v_real[n:]


def make_real_operator_from_complex(H_complex_op):
    N = H_complex_op.shape[0]

    def matvec_real_wrapper(v_real_2n):
        v_c = vec_to_complex(v_real_2n)
        res_c = H_complex_op @ v_c
        return vec_to_real(res_c)

    return sp_linalg.LinearOperator((2 * N, 2 * N), matvec=matvec_real_wrapper, dtype=np.float64)

### Shift invert implementation

In [17]:
def shift_invert_step_jax(H_op_sigma, v_prev, shift=1e-4, tol=1e-9, N_power=3, tol_exit=1e-7):
    v_prev = v_prev / np.linalg.norm(v_prev)
    E_current = np.vdot(v_prev,H_op_sigma(0)@v_prev)
    v_current = v_prev
    sigma = E_current + shift

    scale_factor = -1.0 / shift

    for _ in range(N_power):
        iterations.append(0)
        w, exit_code = sp_linalg.minres(
                H_op_sigma(sigma), v_current, x0=v_current * scale_factor, rtol=tol
            )

        norm_w = np.linalg.norm(w)

        v_new = w / norm_w

        E_new = np.vdot(v_new, H_op_sigma(0) @ v_new).real

        v_current = v_new
        E_current = E_new
        sigma = E_current + shift

        residu = np.linalg.norm(H_op_sigma(E_current)@v_current)

        if residu < tol_exit:
            return E_new, v_new


    return E_current, v_current


In [10]:
def get_branch_jax(H_op_wrapper, E_guess, v_prev, param_list, shift=1e-4, tol=1e-9, N_power=3, tol_exit=1e-7):
    n_steps = len(param_list)
    energies = np.zeros(n_steps, dtype=np.float64)
    vectors = []

    v_current = np.concatenate((v_prev.real, v_prev.imag))

    E_current = E_guess
    energies[0] = E_current
    vectors.append(v_prev)

    for i in tqdm(range(1, n_steps)):
        phi_val = param_list[i]
        H_op_sigma = lambda s: H_op_wrapper(phi_val, s)

        E_current, v_current = shift_invert_step_jax(
            H_op_sigma, v_prev = v_current, shift=shift, tol=tol, N_power=N_power, tol_exit=tol_exit
        )

        energies[i] = E_current
        vectors.append(vec_to_complex(v_current))

    return energies, np.array(vectors)

### Sequential

The following functions are useful to get the shift invert results in a sequential approach. But parallelism is more efficient

### Parallelism

As every branch is independant we can try to parallelise the computation, here is the code

### Classical Lanczos implementation

In [11]:
def get_values_Lanczos(H_func, n_states, param_list, tol=1e-8, initial_guess=None):
    global residu_lanczos
    H_0 = H_func(param_list[0])
    if not initial_guess:
        E_initial, V_initial = sp_linalg.eigsh(H_0, k=n_states, which="SA")
        idx = E_initial.argsort()
        E_initial = E_initial[idx]
        V_initial = V_initial[:, idx]
    else:
        E_initial, V_initial = initial_guess

    n_steps = len(param_list)
    all_energies = np.zeros((n_steps, n_states), dtype=np.float64)
    all_vectors = []

    all_energies[0, :] = E_initial
    all_vectors.append(V_initial)

    for i in range(1, n_steps):
        H = H_func(param_list[i])

        v0_guess = np.mean(all_vectors[-1], axis=1)

        H = sp_linalg.LinearOperator(H.shape, matvec=IterationCounter(H))

        energies_i, vectors_i = sp_linalg.eigsh(
            H, k=n_states, v0=v0_guess, which="SA", tol=tol, ncv=max(10, 2 * n_states + 1)
        )

        idx = energies_i.argsort()
        energies_i = energies_i[idx]
        vectors_i = vectors_i[:, idx]

        all_energies[i, :] = energies_i
        all_vectors.append(vectors_i)

        norms = np.linalg.norm(H @ vectors_i - vectors_i * energies_i, axis=0)
        residu_lanczos.append(np.mean(norms))

    return all_energies, np.array(all_vectors)


# Same code as before but without taking the previous vector as a guess for the next one
def get_values_Lanczos_no_guess(H_func, n_states, param_list, tol=1e-8):
    H_0 = H_func(param_list[0])
    E_initial, V_initial = sp_linalg.eigsh(H_0, k=n_states, which="SA")

    idx = E_initial.argsort()
    E_initial = E_initial[idx]
    V_initial = V_initial[:, idx]

    n_steps = len(param_list)
    all_energies = np.zeros((n_steps, n_states), dtype=np.float64)
    all_vectors = []

    all_energies[0, :] = E_initial
    all_vectors.append(V_initial)

    for i in range(1, n_steps):
        H = H_func(param_list[i])

        H = sp_linalg.LinearOperator(H.shape, matvec=IterationCounter(H))

        energies_i, vectors_i = sp_linalg.eigsh(H, k=n_states, which="SA", tol=tol, ncv=max(100, 2 * n_states + 1))

        idx = energies_i.argsort()
        energies_i = energies_i[idx]
        vectors_i = vectors_i[:, idx]

        all_energies[i, :] = energies_i
        all_vectors.append(vectors_i)

        norms = np.linalg.norm(H @ vectors_i - vectors_i * energies_i, axis=0)
        residu_lanczos.append(np.mean(norms))

    return all_energies, np.array(all_vectors)


class IterationCounter:
    def __init__(self, mat):
        self.mat = mat
        self.count = 0

    def __call__(self, x):
        self.count += 1
        return self.mat @ x


class IterationCounter:
    def __init__(self, mat):
        self.mat = mat
        self.count = 0

    def __call__(self, x):
        self.count += 1
        return self.mat @ x


# Kite application

In [12]:
%load_ext autoreload
%autoreload 2

from common import kite_spectrum_jax_vec as vkite
import jax

jax.config.update("jax_enable_x64", True) 

import jax.numpy as jnp
from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We have to define a scipy linealg operator from the kite to give it to our shift invert

In [13]:
@partial(jax.jit, static_argnames=("dims", "n_g", "phiext_val"))
def global_core_matvec(v_jax_2d, phiext_val, sigma, n_g, ops, dims):
    Hv = vkite.apply_qr_hamitonian_to_vec(ops, dims, v_jax_2d, n_g, phiext=phiext_val)
    return Hv - sigma * v_jax_2d



def get_shifted_kite_operator_real(phiext_val, ops, dims, n_g, sigma):
    """
    Build a real-valued SciPy LinearOperator representing (H - sigma * I),
    where H is applied through a JAX-jitted matrix-vector product.

    The operator maps a real vector of size 2N to 2N, corresponding to the
    real and imaginary parts of a complex vector in C^N. This allows the use
    of real-valued Krylov solvers such as MINRES while keeping the Hamiltonian
    application fully in JAX.

    Parameters
    ----------
    phiext_val : float
        External flux parameter entering the Hamiltonian.
    ops : object
        Precomputed operator structures used to apply the Hamiltonian.
    dims : tuple
        Hilbert space dimensions.
    n_g : float
        Offset charge parameter.
    sigma : float
        Spectral shift defining the operator (H - sigma I).
    """

    dims_tuple = tuple(dims)
    complex_dim = np.prod(dims)
    real_dim = 2 * complex_dim

    def matvec_real(v_real_numpy):
        v_complex = v_real_numpy[:complex_dim] + 1j * v_real_numpy[complex_dim:]
        v_jax_2d = jnp.array(v_complex)[:, None]
        
        res_2d = global_core_matvec(v_jax_2d, phiext_val, sigma, n_g, ops, dims_tuple)
        
        res_complex = np.array(res_2d.ravel(), dtype=np.complex128)
        return np.concatenate((res_complex.real, res_complex.imag))

    return sp_linalg.LinearOperator(
        shape=(real_dim, real_dim),
        matvec=matvec_real,
        dtype=np.float64
    )


# Benchmark

Used parameters on the kite 

In [107]:
print(vkite.DEFAULT_DIMS)

(19, 32, 32, 5)


In [46]:
parameters = vkite.ParamType(
    ECs_GHz=0.072472,
    EL_GHz=1.269,
    ECJ_GHz=4.9895,
    EJ_GHz=17.501,
    eps=0.05702,
    ECc_GHz=0.003989,
    f_r_GHz=4.337,
)


dims = vkite.DEFAULT_DIMS
dims = tuple((np.array(dims) // 1.6).astype(int))
print(f'dimension Hilbert space = {dims}')

ops = vkite.build_static_operators(dims = dims, **parameters)

def my_wrapper(phi, sigma):
    return get_shifted_kite_operator_real(phiext_val=phi, ops=ops, dims=dims, n_g=0.0, sigma=sigma)

dimension Hilbert space = (np.int64(11), np.int64(19), np.int64(19), np.int64(3))


### Benchmark shift value

In [90]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

# --- 1. SETUP: HIGH-RESOLUTION REFERENCE ---
# Zooming in on the critical crossing area
flux_bench = np.linspace(1.565, 1.605, 10)

dim_kept = 8
idx_upper = dim_kept - 1 # Branch 7
idx_lower = dim_kept - 2 # Branch 6

# Storage arrays for the REFERENCE (Ground Truth)
ref_evals_bench = np.zeros((len(flux_bench), dim_kept))
ref_evecs_bench = np.zeros((len(flux_bench), np.prod(dims), dim_kept), dtype=complex)

print(f"1. Computing QR Reference on the crossing zone ({len(flux_bench)} flux points)...")

for i, flux in enumerate(tqdm(flux_bench)):
    ev, ek, _ = vkite.get_qr_esys_with_jac(
        dims=dims, dim_kept=dim_kept, n_g=0.0, phiext=float(flux),
        **parameters, output_jac_params=tuple(),
    )
    ref_evals_bench[i] = ev
    ref_evecs_bench[i] = ek

print("   -> Reference computed. Ready for benchmark.")

1. Computing QR Reference on the crossing zone (10 flux points)...


100%|██████████| 10/10 [00:57<00:00,  5.71s/it]

   -> Reference computed. Ready for benchmark.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
from tqdm import tqdm

# --- 1. SETUP ---
tol_values = [1e-11, 1e-14]
n_power_values = [1, 3, 5]

res_var_si_max = np.zeros((len(tol_values), len(n_power_values)))
res_var_ref_max = np.zeros((len(tol_values), len(n_power_values)))

print(f"\n2. Debug Variance (CORRIGÉ): SI vs Reference...")

# --- FONCTION DE VARIANCE CORRIGÉE ---
def get_max_variance_FIXED(Es, Vs, flux_vals, is_ref=False):
    """Calcule la variance max en reconstruisant les BONS opérateurs."""
    max_var = 0.0
    
    # 1. ON RECONSTRUIT LES OPÉRATEURS ICI (CRUCIAL)
    # On utilise les 'dims' et 'parameters' globaux actuels
    # Cela garantit que H correspond exactement à la physique du moment
    local_ops = vkite.build_static_operators(dims, **parameters)
    
    for i, flux in enumerate(flux_vals):
        if is_ref:
            v_vec = Vs[i]
            e_val = Es[i]
        else:
            v_vec = np.array(Vs[i])
            e_val = Es[i]

        # 2. Normalisation
        v_vec = v_vec.reshape(-1, 1)
        norm = np.linalg.norm(v_vec)
        if norm > 1e-9:
            v_vec = v_vec / norm
        
        # 3. Application Hamiltonien (avec local_ops !)
        hv = vkite.apply_qr_hamitonian_to_vec(
            local_ops, dims, v_vec, n_g=0.0, phiext=float(flux)
        )
        
        # 4. Résidu
        resid = hv - e_val * v_vec
        var = np.linalg.norm(resid)**2
        
        if var > max_var:
            max_var = var
            
    return max_var


# --- BOUCLE ---
for t_idx, tol in enumerate(tqdm(tol_values, desc="Tol Loop")):
    for n_idx, npow in enumerate(n_power_values):
        
        # Init
        E_init = ref_evals_bench[0, idx_upper]
        v_init = ref_evecs_bench[0, :, idx_upper]
        
        # Run Shift-Invert
        Es, Vs = get_branch_jax(
            my_wrapper, E_init, v_init, flux_bench, 
            shift=fixed_shift, tol=tol, tol_exit=1e-10, N_power=npow
        )
        
        # A. Variance SI
        res_var_si_max[t_idx, n_idx] = get_max_variance_FIXED(Es, Vs, flux_bench, is_ref=False)
        
        # B. Variance Ref
        Es_ref_branch = ref_evals_bench[:, idx_upper]
        Vs_ref_branch = ref_evecs_bench[:, :, idx_upper]
        res_var_ref_max[t_idx, n_idx] = get_max_variance_FIXED(Es_ref_branch, Vs_ref_branch, flux_bench, is_ref=True)


# --- PLOT ---
fig, ax = plt.subplots(figsize=(10, 6))
colors = cm.plasma(np.linspace(0, 0.9, len(tol_values)))

for t_idx, tol in enumerate(tol_values):
    # SI (Ligne pleine)
    ax.plot(n_power_values, res_var_si_max[t_idx, :], 
            color=colors[t_idx], marker='o', linestyle='-', linewidth=2,
            label=f'SI Max Var ($\epsilon=10^{{{int(np.log10(tol))}}}$)')
    
    # Ref (Pointillés - DOIT ÊTRE BAS MAINTENANT)
    ax.plot(n_power_values, res_var_ref_max[t_idx, :], 
            color=colors[t_idx], marker='x', linestyle='--', linewidth=1.5, alpha=0.6,
            label=f'Ref Max Var ($\epsilon=10^{{{int(np.log10(tol))}}}$)')

ax.set_ylabel(r'Max Variance $||(H-E)v||^2$')
ax.set_xlabel('RQI Power Iterations')
ax.set_yscale('log')
ax.set_title('Variance Comparison: Shift-Invert vs Reference (Fixed Ops)')
ax.grid(True, which="both", alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

Energie Ref (QR)    : -1.2268279097483334
Energie Calc (ops)  : -1.2815284569551983
Différence          : 0.0547005472068649


### Benchmark n_power and epsilon

In [129]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
from tqdm import tqdm
from jax.numpy.linalg import norm as jnorm

# --- SETUP ---
flux_bench = np.linspace(1.605, 1.645, 10) # Zone réduite
dim_kept = 8
idx_upper = dim_kept - 1 
idx_lower = dim_kept - 2
fixed_shift = 1e-6

# --- REFERENCE CALCULATION ---
print("1. Computing Ground Truth Reference...")
ref_evals_bench = np.zeros((len(flux_bench), dim_kept))
# On a besoin des énergies de ref pour calculer l'erreur
for i, flux in enumerate(flux_bench):
    ev, _, _ = vkite.get_qr_esys_with_jac(
        dims=dims, dim_kept=dim_kept, n_g=0.0, phiext=float(flux),
        **parameters, output_jac_params=tuple(),
    )
    ref_evals_bench[i] = ev

# --- HELPER: VARIANCE CALCULATION ---
def compute_max_trajectory_variance(Es, Vs, flux_vals):
    """Calcule max(||(H-E)v||^2) sur toute la trajectoire."""
    max_var = 0.0
    
    for i, flux in enumerate(flux_vals):
        e_val = Es[i]
        # Reshape (N,) -> (N, 1) pour vkite
        v_vec = np.array(Vs[i]).reshape(-1, 1) 
        
        # H*v
        hv = vkite.apply_qr_hamitonian_to_vec(
            ops, dims, v_vec, n_g=0.0, phiext=float(flux)
        )
        
        # Résidu r = Hv - Ev
        resid = hv - e_val * v_vec
        
        # Variance = ||r||^2 (si v est normé)
        # On utilise la norme L2 standard
        var = np.linalg.norm(resid)**2
        
        if var > max_var:
            max_var = var
            
    return max_var

print("   -> Reference & Helpers ready.")

1. Computing Ground Truth Reference...
   -> Reference & Helpers ready.


In [130]:
# --- PARAMETERS ---
tol_values = [1e-11,1e-14]
n_power_values = [1,3,5]

# Storage: results[tol_idx][npow_idx]
res_time = np.zeros((len(tol_values), len(n_power_values)))
res_energy_err = np.zeros((len(tol_values), len(n_power_values)))
res_variance = np.zeros((len(tol_values), len(n_power_values)))

print(f"\n2. Launching Grid Benchmark (Tol x N_power)...")

# Boucle principale sur les tolérances (Couleurs)
for t_idx, tol in enumerate(tqdm(tol_values, desc="Tolerance Loop")):
    
    # Boucle interne sur N_power (Axe X)
    for n_idx, npow in enumerate(n_power_values):
        
        t0 = time.time()
        
        # On part toujours de la branche du HAUT (Upper)
        # Shift-Invert suivra le chemin Diabatique (vers le bas) à cause du shift 1e-6
        E_init = ref_evals_bench[0, idx_upper]
        
        # NOTE: Pour calculer la variance, il nous faut les vecteurs 'Vs' !
        # Assure-toi que get_branch_jax retourne bien (Es, Vs)
        # On passe tol_exit=tol pour être cohérent
        Es, Vs = get_branch_jax(
            my_wrapper, E_init, v_init, flux_bench, 
            shift=fixed_shift, tol=tol, tol_exit=1e-10, N_power=npow
        )
        
        # A. TIME
        res_time[t_idx, n_idx] = (time.time() - t0) / len(flux_bench) # Avg time per point
        
        # B. ENERGY ERROR (Agnostic / Nearest Branch)
        # Puisque shift=1e-6 est Diabatique, on risque de croiser.
        # On compare à la ref la plus proche point par point.
        dist_ref7 = np.abs(np.array(Es) - ref_evals_bench[:, idx_upper])
        dist_ref6 = np.abs(np.array(Es) - ref_evals_bench[:, idx_lower])
        min_dist = np.minimum(dist_ref7, dist_ref6)
        res_energy_err[t_idx, n_idx] = np.max(min_dist)
        
        # C. VARIANCE
        # On calcule la variance max sur la trajectoire
        res_variance[t_idx, n_idx] = compute_max_trajectory_variance(Es, Vs, flux_bench)

print("Benchmark completed.")


2. Launching Grid Benchmark (Tol x N_power)...


Tolerance Loop: 100%|██████████| 2/2 [03:01<00:00, 90.73s/it]

Benchmark completed.


In [131]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# --- PLOTTING ---
# Colormap pour les tolérances
colors = cm.turbo(np.linspace(0, 0.9, len(tol_values)))

# === FIG 1: PERFORMANCE (TIME) ===
fig1, ax1 = plt.subplots(figsize=(10, 6))

for t_idx, tol in enumerate(tol_values):
    # J'ai retiré l'argument 'marker=...'
    ax1.plot(n_power_values, res_time[t_idx, :], 
             color=colors[t_idx], linewidth=2,
             label=f'$\\epsilon = 10^{{{int(np.log10(tol))}}}$')

ax1.set_xlabel('RQI Power Iterations ($N_{power}$)')
ax1.set_ylabel('Avg Time per Flux Point (s)')
ax1.set_title('Performance Benchmark: Computation Cost')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.grid(True, which="both", alpha=0.3)
ax1.legend(title="Tolerance (tol)", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# === FIG 2: ACCURACY (ENERGY & VARIANCE) ===
fig2, (ax2, ax3) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

# Subplot 1: Energy Error
for t_idx, tol in enumerate(tol_values):
    # J'ai retiré l'argument 'marker=...'
    ax2.plot(n_power_values, res_energy_err[t_idx, :], 
             color=colors[t_idx], linewidth=1.5,
             label=f'$\\epsilon = 10^{{{int(np.log10(tol))}}}$')

ax2.set_ylabel(r'Max Energy Error: $||E - E_{ref}||_\infty$')
ax2.set_title(f'Accuracy Benchmark: Energy Deviation (Shift={fixed_shift})')
ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.grid(True, which="both", alpha=0.3)
ax2.legend(title="Tolerance", bbox_to_anchor=(1.05, 1), loc='upper left')

# Subplot 2: Variance
for t_idx, tol in enumerate(tol_values):
    # J'ai retiré l'argument 'marker=...'
    ax3.plot(n_power_values, res_variance[t_idx, :], 
             color=colors[t_idx], linestyle='--', linewidth=1.5,
             label=f'$\\epsilon = 10^{{{int(np.log10(tol))}}}$')

ax3.set_ylabel(r'Max Variance: $||(H^2) - \langle H \rangle^2||_\infty$')
ax3.set_xlabel('RQI Power Iterations ($N_{power}$)')
ax3.set_title('Accuracy Benchmark: Hamiltonian Variance')
ax3.set_yscale('log')
ax3.grid(True, which="both", alpha=0.3)

plt.tight_layout()
plt.show()

In [137]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
from tqdm import tqdm

# --- 1. SETUP ---
tol_values = [1e-11, 1e-14]
n_power_values = [1, 3, 5]

res_var_si_max = np.zeros((len(tol_values), len(n_power_values)))
res_var_ref_max = np.zeros((len(tol_values), len(n_power_values)))

print(f"\n3. Debug Variance (INTRINSÈQUE): SI vs Reference...")

# --- FONCTION DE VARIANCE INTRINSÈQUE ---
def get_intrinsic_variance(Vs, flux_vals, is_ref=False):
    """
    Calcule la variance SANS utiliser l'énergie de référence.
    On projette H sur v pour trouver l'énergie propre au vecteur.
    """
    max_var = 0.0
    
    # 1. On reconstruit les opérateurs (Toujours nécessaire)
    local_ops = vkite.build_static_operators(dims, **parameters)
    
    for i, flux in enumerate(flux_vals):
        if is_ref:
            v_vec = Vs[i]
        else:
            v_vec = np.array(Vs[i])

        # 2. Normalisation (CRUCIAL)
        v_vec = v_vec.reshape(-1, 1)
        norm = np.linalg.norm(v_vec)
        if norm > 1e-9:
            v_vec = v_vec / norm
        
        # 3. Application Hamiltonien
        hv = vkite.apply_qr_hamitonian_to_vec(
            local_ops, dims, v_vec, n_g=0.0, phiext=float(flux)
        )
        
        # 4. CALCUL DE L'ÉNERGIE LOCALE (La clé du succès !)
        # E_local = <v | H | v>
        # C'est l'énergie "vue" par cet opérateur précis
        e_local = np.vdot(v_vec, hv).real[0, 0] # .item() si scalaire
        
        # 5. Résidu Intrinsèque
        resid = hv - e_local * v_vec
        var = np.linalg.norm(resid)**2
        
        if var > max_var:
            max_var = var
            
    return max_var


# --- BOUCLE ---
for t_idx, tol in enumerate(tqdm(tol_values, desc="Tol Loop")):
    for n_idx, npow in enumerate(n_power_values):
        
        E_init = ref_evals_bench[0, idx_upper]
        v_init = ref_evecs_bench[0, :, idx_upper]
        
        # Run SI
        Es, Vs = get_branch_jax(
            my_wrapper, E_init, v_init, flux_bench, 
            shift=fixed_shift, tol=tol, tol_exit=1e-10, N_power=npow
        )
        
        # Note: On ne passe plus Es, car on le recalcule en interne
        res_var_si_max[t_idx, n_idx] = get_intrinsic_variance(Vs, flux_bench, is_ref=False)
        
        # Ref
        Vs_ref_branch = ref_evecs_bench[:, :, idx_upper]
        res_var_ref_max[t_idx, n_idx] = get_intrinsic_variance(Vs_ref_branch, flux_bench, is_ref=True)


# --- PLOT ---
fig, ax = plt.subplots(figsize=(10, 6))
colors = cm.plasma(np.linspace(0, 0.9, len(tol_values)))

for t_idx, tol in enumerate(tol_values):
    # SI
    ax.plot(n_power_values, res_var_si_max[t_idx, :], 
            color=colors[t_idx], marker='o', linestyle='-', linewidth=2,
            label=f'SI Intrinsic Var ($\epsilon=10^{{{int(np.log10(tol))}}}$)')
    
    # Ref
    ax.plot(n_power_values, res_var_ref_max[t_idx, :], 
            color=colors[t_idx], marker='x', linestyle='--', linewidth=1.5, alpha=0.6,
            label=f'Ref Intrinsic Var ($\epsilon=10^{{{int(np.log10(tol))}}}$)')

ax.set_ylabel(r'Intrinsic Variance $||(H - \langle H \rangle)v||^2$')
ax.set_xlabel('RQI Power Iterations')
ax.set_yscale('log')
ax.set_title('Intrinsic Accuracy Check (Offset Independent)')
ax.grid(True, which="both", alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()


3. Debug Variance (INTRINSÈQUE): SI vs Reference...


Tol Loop:   0%|          | 0/2 [00:19<?, ?it/s]


IndexError: invalid index to scalar variable.

### Benchmark tol_exit

### Benchmark reference vs shift invert one energy 

In [113]:
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- 1. CONFIGURATION ---
base_dims = np.array([19, 32, 32, 5])
flux_prev = 1.596
flux_curr = 1.600
divisors = [2.05, 1.75, 1.35, 1.12, 1.0]
divisors = [2.05, 21.9]

# Paramètres Shift-Invert
dim_kept = 8
target_idx = dim_kept - 1
best_shift = 1e-6
best_tol = 1e-12
best_npow = 5

# Stockage
results = {
    "hilbert_size": [],
    "time_ref": [],
    "time_si": [],
    "fidelity": [],
    "variance_si": [],  # Variance du Shift-Invert
    "variance_ref": []  # Variance de la Référence
}

print(f"⚔️ SCALING TEST : DUEL COMPLET (Temps & Précision) ⚔️")

for div in tqdm(divisors, desc="Simulations"):
    # --- Dimensions ---
    raw_dims = (base_dims / div).astype(int)
    # Force dimension impaire + minimum 3
    curr_dims = np.where(raw_dims % 2 == 0, raw_dims + 1, raw_dims)
    curr_dims = np.maximum(curr_dims, 3)
    
    hilbert_size = np.prod(curr_dims)
    curr_dims_tuple = tuple(curr_dims)
    
    results["hilbert_size"].append(hilbert_size)
    
    # --- Phase 0: Guess ---
    ev_prev, ek_prev, _ = vkite.get_qr_esys_with_jac(
        dims=curr_dims_tuple, dim_kept=dim_kept, n_g=0.0, phiext=flux_prev,
        **parameters, output_jac_params=tuple()
    )
    E_guess = ev_prev[target_idx]
    v_guess = ek_prev[:, target_idx]
    
    # --- Phase 1: Reference (QR) ---
    t0 = time.time()
    ev_ref, ek_ref, _ = vkite.get_qr_esys_with_jac(
        dims=curr_dims_tuple, dim_kept=dim_kept, n_g=0.0, phiext=flux_curr,
        **parameters, output_jac_params=tuple()
    )
    results["time_ref"].append(time.time() - t0)
    
    # Données pour comparaison
    v_ref_true = ek_ref[:, target_idx]
    E_ref_true = ev_ref[target_idx]
    
    # --- Phase 2: Shift-Invert ---
    t0 = time.time()
    Es_si, Vs_si = get_branch_jax(
        my_wrapper, E_guess, v_guess, [flux_curr], 
        shift=best_shift, tol=best_tol, tol_exit=best_tol, N_power=best_npow
    )
    results["time_si"].append(time.time() - t0)
    
    v_si_final = np.array(Vs_si[0])
    E_si_final = Es_si[0]
    
    # --- Phase 3: Calculs de Qualité ---
    
    # A. Fidélité
    fid = np.abs(np.vdot(v_si_final, v_ref_true))**2
    results["fidelity"].append(fid)
    
    # B. Variances (Ref vs SI)
    # 1. Reconstruction des opérateurs pour la dimension actuelle
    ops_curr = vkite.build_static_operators(curr_dims_tuple, **parameters)
    
    # Fonction helper locale pour calculer la variance
    def get_variance(vec, energy, ops, dims):
        v_col = vec.reshape(-1, 1)
        hv = vkite.apply_qr_hamitonian_to_vec(ops, dims, v_col, n_g=0.0, phiext=flux_curr)
        resid = hv - energy * v_col
        return np.linalg.norm(resid)**2

    # Variance Shift-Invert
    var_si = get_variance(v_si_final, E_si_final, ops_curr, curr_dims_tuple)
    results["variance_si"].append(var_si)
    
    # Variance Référence (Pour comparer)
    var_ref = get_variance(v_ref_true, E_ref_true, ops_curr, curr_dims_tuple)
    results["variance_ref"].append(var_ref)


# --- PLOT FINAL (3 Panneaux) ---
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 14), sharex=True)

# 1. TEMPS
ax1.loglog(results["hilbert_size"], results["time_ref"], 's-', color='grey', label='Reference (QR)')
ax1.loglog(results["hilbert_size"], results["time_si"], 'o-', color='tab:red', label='Shift-Invert')

speedup = results["time_ref"][-1] / results["time_si"][-1]
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
ax1.text(results["hilbert_size"][-1], results["time_si"][-1], 
         f"Speedup: {speedup:.1f}x", fontsize=12, verticalalignment='bottom', horizontalalignment='right', bbox=props)
ax1.set_ylabel('Time (s)')
ax1.set_title('1. Performance Scaling')
ax1.grid(True, which="both", alpha=0.3)
ax1.legend()

# 2. FIDÉLITÉ
ax2.plot(results["hilbert_size"], results["fidelity"], 'd-', color='tab:green')
ax2.set_ylabel('Adiabatic Fidelity')
ax2.set_ylim(0.999, 1.0001)
ax2.set_title('2. Agreement with Reference')
ax2.grid(True, alpha=0.3)

# 3. VARIANCE (Le plot demandé)
ax3.plot(results["hilbert_size"], results["variance_ref"], 's--', color='grey', label='Reference Variance')
ax3.plot(results["hilbert_size"], results["variance_si"], 'o-', color='tab:orange', label='Shift-Invert Variance')

ax3.set_ylabel('Residual Variance $||(H-E)v||^2$')
ax3.set_xlabel('Hilbert Space Dimension (N)')
ax3.set_yscale('log')
ax3.set_title('3. Intrinsic Physical Accuracy')
ax3.legend()
ax3.grid(True, which="both", alpha=0.3)

plt.tight_layout()
plt.show()

⚔️ SCALING TEST : DUEL COMPLET (Temps & Précision) ⚔️


0it [00:00, ?it/s]          | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]█████     | 1/2 [00:04<00:04,  4.03s/it]
Simulations: 100%|██████████| 2/2 [00:10<00:00,  5.24s/it]
